## Prepared Work

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import os
import json
import time
import re
from selenium import webdriver

cookiestr = 'aws-account-alias=bb102; x-wl-uid=1qO1zbLT/VurrTD8olrRaxt4ZvTE+vTeFIHu58TFdpWz5wSlgeDL/v+q5P3gFNGfccOVMboQEwus=; AMCV_4A8581745834114C0A495E2B%40AdobeOrg=-1891778711%7CMCIDTS%7C17459%7CMCMID%7C52253475372251991424690344856724644964%7CMCOPTOUT-1508403795s%7CNONE%7CMCAID%7C2CF30EF7852A888A-40000300E0001C2D%7CvVersion%7C2.4.0; s_nr=1508396596374-New; s_lv=1508396596375; skin=noskin; session-token=0lQkzegQRtwPu7GiPFEAEDYLCK3p6WW8vF/g2Kr93EnMHKmH2uTgd30RM43tJCAw2uo2xeco+fz9qCC4dzTpW86EX5YF5lVFvMOXhSncgNmhfgnG7DaXuUwvxrXhAtluWYQgfWI8I22bxDO6KWvBDwfM6IFCfsYouZ6kt3oXTHzy6wSehl3qokrzeEBzLpfPu6mlixQc/Yuommb6ztr0yiL/6dzBf5UjFImpI3OXvIkvwGuyBITnHJH9epKv+Lec; amznacsleftnav-74393fbe-66a6-3a52-840b-37b54d8c76ce=1,21,3; ubid-main=132-7315371-9843811; session-id-time=2082787201l; session-id=135-5046157-5863552; csm-hit=85CP8ATFWH4EXAFXQX3Q+s-M5199TSF54AFVK1M6RJW|1509623128403'
cookies = cookiestr.split(";")
cookiedata ={}
for cookie in cookies:
    cookiedata[cookie.split("=")[0]]=cookie.split("=")[1]

ssession = requests.Session()
headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit 537.36 (KHTML, like Gecko) Chrome","Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"}

## 設定驅動器

In [2]:
# 使用chrome驅動器
browser = webdriver.Chrome()
# 每一個網頁不超過30秒
browser.set_page_load_timeout(30)
browser.get('https://www.amazon.com/Best-Sellers-Toys-Games-Dress-Up-Pretend-Play/zgbs/toys-and-games/166316011/ref=t#1')

## 抓頁碼

In [3]:
page_info = browser.find_element_by_css_selector('#zg_page5 > a')
print(page_info)
print(page_info.text)
pages = page_info.text.split('-')[1]
pages = int(pages)
pages = int(pages/20)
pages

<selenium.webdriver.remote.webelement.WebElement (session="94167cdb3ba3bb29daa148d1bb6dcbce", element="0.9389631749523832-1")>
81-100


5

## 抓取href_list

In [75]:
start = time.time()
Amzlist=[]
for i in range(pages):
    if i > 2:
        break
    print('第%d頁' % (i+1))
    url ='https://www.amazon.com/Best-Sellers-Toys-Games-Dress-Up-Pretend-Play/zgbs/toys-and-games/166316011/ref=t#'+ str(i+1)
    browser.get(url)
    #windows.scrollTo(x座標,y座標)=>讓他執行滾動
    browser.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    time.sleep(3)
    goods = browser.find_element_by_css_selector('#zg_left_col1 > #zg_centerListWrapper')
#     print(goods.text)
#     print('第%d頁有%d件商品' %((i+1),len(goods.text)))
    all_goods = goods.find_elements_by_class_name('zg_itemImmersion')
    try:
        href = [g.find_element_by_css_selector('div.zg_itemWrapper > div > a').get_attribute('href')for g in all_goods]
        Amzlist += href
        time.sleep(0.1)
    except Exception as e:
            print(e)
            print("none")
            pass
end = time.time() 

print("-----------------")
Amzlist="\n".join(Amzlist) 
# print(Amzlist)
print(end-start)




第1頁
第2頁
第3頁
-----------------
13.55503797531128


## 轉成txt

In [5]:
if not os.path.exists('../20180118/US/'):
    os.makedirs('../20180118/US/')

In [76]:
with open('../20180118/US/Dressing Up & Costumes.txt','w',encoding='utf-8') as f :
        f.write(Amzlist)

## 抓ReviewList

In [77]:
start = time.time()
Rlist=[]
none = 0
for i in range(pages):
    if i > 2:
        break
    print('第%d頁' % (i+1))
    url ='https://www.amazon.com/Best-Sellers-Toys-Games-Dress-Up-Pretend-Play/zgbs/toys-and-games/166316011/ref=t#'+ str(i+1)
    browser.get(url)
    #windows.scrollTo(x座標,y座標)=>讓他執行滾動
    browser.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    time.sleep(3)
    goods = browser.find_element_by_css_selector('#zg_left_col1 > #zg_centerListWrapper')
#     print(goods.text)
#     print('第%d頁有%d件商品' %((i+1),len(goods.text)))
    all_goods = goods.find_elements_by_class_name('zg_itemImmersion')
    for g in all_goods:
        try:
            reviewlist = [g.find_element_by_class_name('a-size-small').get_attribute('href')]
            Rlist += reviewlist 
            time.sleep(0.1)
        except Exception as e:
            none += 1
            
#                 print(e)
#                 print("none")
            pass
end = time.time() 
print(none)
print("-----------------")
Rlist="\n".join(Rlist) 
# print(Rlist)
print(end-start)

第1頁
第2頁
第3頁
0
-----------------
17.075826168060303


## ReviewList轉成txt

In [8]:
if not os.path.exists('../../Review_list/20180118/US/'):
    os.makedirs('../../Review_list/20180118/US/')

In [78]:
with open('../../Review_list/20180118/US/Dressing Up & Costumes_review.txt','w',encoding='utf-8') as f :
        f.write(Rlist)

## 將 contents 存成 csv檔

In [79]:
start = time.time()
conlist=[]
for i in range(pages):
    if i > 2:
        break
    print('第%d頁' % (i+1))
    url ='https://www.amazon.com/Best-Sellers-Toys-Games-Dress-Up-Pretend-Play/zgbs/toys-and-games/166316011/ref=t#'+ str(i+1)
    browser.get(url)
    #windows.scrollTo(x座標,y座標)=>讓他執行滾動
    browser.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    time.sleep(3)
    goods = browser.find_element_by_css_selector('#zg_left_col1 > #zg_centerListWrapper')
#     print(goods.text)
#     print('第%d頁有%d件商品' %((i+1),len(goods.text)))
    all_goods = goods.find_elements_by_class_name('zg_itemImmersion')
    for g in all_goods:
        try:
            contents = [g.find_element_by_class_name('zg_rankNumber').text+ g.find_element_by_class_name('a-link-normal').text+";"\
                    +g.find_element_by_css_selector('div.zg_itemWrapper > div > div.a-icon-row > a').get_attribute('title')+";"\
                    +g.find_element_by_class_name('a-size-small').text+";"\
                    +g.find_element_by_css_selector('div.zg_itemWrapper > div > div.a-row > span.a-size-base.a-color-price > span').text+";"\
                   +g.find_element_by_css_selector('div.zg_itemWrapper > div > a').get_attribute('href')]
            conlist += contents
        except Exception as e:
            print(g.find_element_by_class_name('zg_rankNumber').text + g.find_element_by_class_name('a-link-normal').text)
            pass
        
end = time.time() 

print("-----------------")
print(end-start)
conlist="\n".join(conlist) 
# print(conlist)

第1頁
第2頁
第3頁
-----------------
20.869002103805542


In [80]:
with open('../20180118/US/Dressing Up & Costumes.csv','w',encoding='utf-8') as f :
        f.write(conlist)

## readlines()

In [35]:
with open('../../Extract/20171214/US/Baby & Toddler Toys.txt',encoding='utf-8') as f :
    a=f.readlines()

In [36]:
hrefs=[i.split("\n")[0] for i in a]

In [37]:
hrefs[0]

'https://www.amazon.com/Nuby-Octopus-Hoopla-Bathtime-Purple/dp/B008CDR7LW/ref=zg_bs_196601011_1?_encoding=UTF8&psc=1&refRID=R8WD6SZYZXF6SH2DGZBZ'

## 轉成100個html檔

In [38]:
import requests
from bs4 import BeautifulSoup
# 測試網頁是否正常連接
session = requests.Session()
headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit 537.36 (KHTML, like Gecko) Chrome","Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"}
res = requests.get(hrefs[0])
res

<Response [503]>

In [39]:
from bs4 import BeautifulSoup
import multiprocessing as mp
import concurrent.futures
import requests
import os


session = requests.Session()
headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit 537.36 (KHTML, like Gecko) Chrome","Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"}
cookiestr = '__cfduid=df4417457002ed1d390cc65a2f7734b9d1494830719; _gat=1; _ga=GA1.2.1333249610.1494830798; _gid=GA1.2.970157078.1502190098'
cookies = cookiestr.split(";")
cookiedata ={}
if not os.path.exists('../../20171214/US/Baby & Toddler Toys/'):
    os.makedirs('../../20171214/US/Baby & Toddler Toys/')
for cookie in cookies:
    cookiedata[cookie.split("=")[0]]=cookie.split("=")[1]
    
import time
errorlist = []
error = 0
start=time.time()
for href in hrefs:
    res=requests.get(href,cookies = cookiedata)
    res.content
    try:
        with open('../../20171214/US/Baby & Toddler Toys/'+href.split('/')[-4]+'.html','w',encoding='utf-8') as f :
                print("==========================file %s=====================" % (href.split('/')[-4]))
                f.write(res.text)
                time.sleep(2)
                if 'Sorry! Something went wrong on our end.' in res.text:
                    error += 1
                    errorlist.append(href)
                else:
                    if 'Type the characters you see in this image:'in res.text:
                        error += 1
                        errorlist.append(href)
                    else:
                        if 'Sorry! Something went wrong on our end.' in res.text:
                            error += 1
                            errorlist.append(href)
                    
                        with open('../../20171214/US/Baby & Toddler Toys_error.txt','w',encoding='utf-8') as f1:
                            for row in errorlist:
                                f1.write(row + '\n')
    except Exception as e:
        print(e)
        time.sleep(0.1)
        pass
    
    
    

end=time.time()
print(end-start)
print(error)

==========================file Nuby-Octopus-Hoopla-Bathtime-Purple=====================
==========================file VTech-KidiBeats-Drum-Frustration-Packaging=====================
==========================file Green-Toys-Ferry-Bathtub-White=====================
==========================file First-Years-Stacking-Up-Cups=====================
==========================file VTech-Stand-Learning-Frustration-Packaging=====================
==========================file Nuby-IcyBite-Keys-Teether-Free=====================
==========================file VTech-Drop-Go-Dump-Truck=====================
==========================file Sassy-80109-Developmental-Bumpy-Ball=====================
==========================file Crayola-Color-Dropz-Ounce-Tablets=====================
==========================file Mega-Bloks-80-Piece-Building-Classic=====================
==========================file Manhattan-Toy-Winkel-Sensory-Teether=====================
==========================file Baby-Einstein-

==========================file Kids-II-81107-Oball-Shaker=====================
==========================file Lamaze-L27024A-Freddie-The-Firefly=====================
==========================file VTech-80-077000-Sit-to-Stand-Learning-Walker=====================
==========================file Bright-Starts-Clack-Slide-Activity=====================
==========================file Boon-Water-Bugs-Floating-Bath=====================
==========================file Galactic-Heroes-Imperial-AT-AT-Fortress=====================
250.78685998916626
97


## Error test

In [40]:
with open('../../20171214/US/Baby & Toddler Toys_error.txt',encoding='utf-8') as f :
    b=f.readlines()

In [41]:
hrefs2=[i.split("\n")[0] for i in b]

In [42]:
hrefs2[0]

'https://www.amazon.com/Nuby-Octopus-Hoopla-Bathtime-Purple/dp/B008CDR7LW/ref=zg_bs_196601011_1?_encoding=UTF8&psc=1&refRID=R8WD6SZYZXF6SH2DGZBZ'

In [44]:
import requests
from bs4 import BeautifulSoup
# 測試網頁是否正常連接
session = requests.Session()
headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit 537.36 (KHTML, like Gecko) Chrome","Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"}
res = requests.get(hrefs2[0])
res

<Response [503]>

In [43]:
from bs4 import BeautifulSoup
import multiprocessing as mp
import concurrent.futures
import requests
import os


session = requests.Session()
headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit 537.36 (KHTML, like Gecko) Chrome","Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"}
cookiestr = '__cfduid=df4417457002ed1d390cc65a2f7734b9d1494830719; _gat=1; _ga=GA1.2.1333249610.1494830798; _gid=GA1.2.970157078.1502190098'
cookies = cookiestr.split(";")
cookiedata ={}
if not os.path.exists('../../20171214/US/Baby & Toddler Toys2/'):
    os.makedirs('../../20171214/US/Baby & Toddler Toys2/')
for cookie in cookies:
    cookiedata[cookie.split("=")[0]]=cookie.split("=")[1]
    
import time
errorlist = []
error = 0
start=time.time()
for href2 in hrefs2:
    res2=requests.get(href2,cookies = cookiedata)
    res2.content
    try:
        with open('../../20171214/US/Baby & Toddler Toys2/'+href2.split('/')[-4]+'.html','w',encoding='utf-8') as f2 :
                print("==========================file %s=====================" % (href2.split('/')[-4]))
                f2.write(res2.text)
                time.sleep(2)
                if 'Sorry! Something went wrong on our end.' in res2.text:
                    error += 1
                    errorlist.append(href2)
                else:
                    if 'Type the characters you see in this image:'in res2.text:
                        error += 1
                        errorlist.append(href2)
                    else:
                        if 'Sorry! Something went wrong on our end.' in res2.text:
                            error += 1
                            errorlist.append(href2)
                    
                        with open('../../20171214/US/Baby & Toddler Toys_error2.txt','w',encoding='utf-8') as f3:
                            for row in errorlist:
                                f3.write(row + '\n')
    except Exception as e:
        print(e)
        time.sleep(0.1)
        pass
    
    
    

end=time.time()
print(end-start)
print(error)

==========================file Nuby-Octopus-Hoopla-Bathtime-Purple=====================
==========================file VTech-KidiBeats-Drum-Frustration-Packaging=====================
==========================file Green-Toys-Ferry-Bathtub-White=====================
==========================file VTech-Stand-Learning-Frustration-Packaging=====================
==========================file Nuby-IcyBite-Keys-Teether-Free=====================
==========================file VTech-Drop-Go-Dump-Truck=====================
==========================file Sassy-80109-Developmental-Bumpy-Ball=====================
==========================file Crayola-Color-Dropz-Ounce-Tablets=====================
==========================file Mega-Bloks-80-Piece-Building-Classic=====================
==========================file Manhattan-Toy-Winkel-Sensory-Teether=====================
==========================file Baby-Einstein-Along-Tunes-Musical=====================
==========================file VTech-Bus